In [6]:
import numpy as np
import re

def generate_time_evolution_circuit(input_qasm_path: str, output_qasm_path: str, t: float):
    """从初态制备QASM文件生成完整的时间演化电路
    Args:
        input_qasm_path: 输入QASM文件路径
        output_qasm_path: 输出QASM文件路径
        t: 演化时间
    """
    # 读取初态制备电路
    with open(input_qasm_path, 'r') as f:
        initial_circuit = f.read()
    
    # 解析初态电路 - 使用更健壮的解析方法
    header = []
    body = []
    for line in initial_circuit.split('\n'):
        line = line.strip()
        if not line or line.startswith('//'):  # 跳过空行和注释
            continue
        if line.startswith('OPENQASM') or line.startswith('include') or line.startswith('qreg'):
            header.append(line)
        else:
            # 使用正则表达式解析门指令
            match = re.match(r'(\w+)\s*\(?([\d\.,\s\-\+]*)\)?\s*([\w\[\],\s]+);?', line)
            if match:
                body.append(line)
    
    # 时间演化电路
    evolution_circuit = []
    n_qubits = 10
    n_x = n_y = 5  # x和y方向各5个量子比特
    
    # 为x和y方向生成演化电路
    for direction, start_q in [('x', 0), ('y', 5)]:
        n_alpha = n_x if direction == 'x' else n_y
        qubits = list(range(start_q, start_q + n_alpha))
        
        # 前向QFT
        evolution_circuit.extend(qft_circuit(qubits, inverse=False))
        
        # 应用exp(-i k_α² t/2)
        evolution_circuit.extend(exp_k_squared_circuit(qubits, t, n_alpha))
        
        # 逆QFT
        evolution_circuit.extend(qft_circuit(qubits, inverse=True))
    
    # 合并所有部分
    full_circuit = '\n'.join(header) + '\n'
    full_circuit += '\n'.join(body) + '\n'
    full_circuit += '\n'.join(evolution_circuit)
    
    # 写入输出文件
    with open(output_qasm_path, 'w') as f:
        f.write(full_circuit)

def qft_circuit(qubits: list, inverse: bool = False) -> list:
    """生成QFT或逆QFT电路
    Args:
        qubits: 作用的量子比特列表
        inverse: 是否为逆QFT
    Returns:
        OpenQASM指令列表
    """
    n = len(qubits)
    instructions = []
    
    if inverse:
        # 逆QFT：从最高位开始
        for i in range(n-1, -1, -1):
            for j in range(n-1, i, -1):
                # 受控相位门 (角度取负)
                lam = -np.pi / (2 ** (j - i))
                instructions.append(f"u(0,0,{lam}) q[{qubits[j]}];")
                instructions.append(f"cz q[{qubits[i]}],q[{qubits[j]}];")
                instructions.append(f"u(0,0,{-lam}) q[{qubits[j]}];")
            instructions.append(f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[i]}];")  # Hadamard分解
    else:
        # 标准QFT：从最低位开始
        for i in range(n):
            instructions.append(f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[i]}];")  # Hadamard分解
            for j in range(i+1, n):
                # 受控相位门
                lam = np.pi / (2 ** (j - i))
                instructions.append(f"u(0,0,{lam}) q[{qubits[j]}];")
                instructions.append(f"cz q[{qubits[i]}],q[{qubits[j]}];")
                instructions.append(f"u(0,0,{-lam}) q[{qubits[j]}];")
    
    return instructions

def exp_k_squared_circuit(qubits: list, t: float, n_alpha: int) -> list:
    """生成exp(-i k_α² t/2)电路
    Args:
        qubits: 作用的量子比特列表 (索引0对应Z_1)
        t: 演化时间
        n_alpha: 方向上的量子比特数
    Returns:
        OpenQASM指令列表
    """
    instructions = []
    
    # 计算系数 (修正后的公式)
    c = []
    # Z_1系数 (修正为2^{n_α-1})
    c.append(2**(n_alpha - 1))  
    # 其他Z_j系数
    for j in range(1, n_alpha):
        c.append(-0.5 * (2 ** (n_alpha - j - 1)))
    
    # 计算单比特旋转角度
    single_angles = []
    for j in range(n_alpha):
        # 角度 = -c_j * t (详见推导说明)
        angle = -c[j] * t
        single_angles.append(angle)
    
    # 应用单量子比特旋转
    for i, angle in enumerate(single_angles):
        instructions.append(f"u(0,0,{angle}) q[{qubits[i]}];")
    
    # 双量子比特项 (交叉项)
    for i in range(n_alpha):
        for j in range(i + 1, n_alpha):
            # 计算Z_i Z_j项系数 (2 * c_i * c_j)
            cross_coeff = 2 * c[i] * c[j]
            angle_zz = cross_coeff * t
            
            # 使用7门序列实现exp(-iφZ_iZ_j)
            # 门序列: H_j - CZ_{i,j} - H_j - Rz_j(2φ) - H_j - CZ_{i,j} - H_j
            instructions.extend([
                f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[j]}];",  # H_j
                f"cz q[{qubits[i]}],q[{qubits[j]}];",
                f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[j]}];",  # H_j
                f"u(0,0,{2*angle_zz}) q[{qubits[j]}];",  # Rz_j(2φ)
                f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[j]}];",  # H_j
                f"cz q[{qubits[i]}],q[{qubits[j]}];",
                f"u(1.5707963267948966,0,3.141592653589793) q[{qubits[j]}];",  # H_j
            ])
    
    return instructions

def parse_qasm(qasm_content: str) -> list:
    """解析QASM内容，返回门指令列表"""
    instructions = []
    for line in qasm_content.split('\n'):
        line = line.strip()
        if not line or line.startswith('//') or line.startswith('OPENQASM') or line.startswith('include') or line.startswith('qreg'):
            continue
        
        # 使用正则表达式解析指令
        match = re.match(r'(\w+)\s*\(?([\d\.,\s\-\+]*)\)?\s*([\w\[\],\s]+);?', line)
        if match:
            gate_type = match.group(1)
            params = match.group(2).split(',') if match.group(2) else []
            qubits = [q.strip() for q in match.group(3).split(',')]
            
            # 构建标准化的QASM行
            if params:
                param_str = ','.join(params)
                instructions.append(f"{gate_type}({param_str}) {' '.join(qubits)};")
            else:
                instructions.append(f"{gate_type} {' '.join(qubits)};")
    
    return instructions


In [7]:
# 示例使用
if __name__ == "__main__":
    # 输入参数
    input_qasm = "diverging_flow_t=0.qasm"  # 初态制备QASM文件
    output_qasm = "diverging_flow_t=pi_over_2_test_4.qasm"  # 输出完整电路QASM文件
    
    t = np.pi / 2  # 演化时间
    
    # 生成完整电路
    generate_time_evolution_circuit(input_qasm, output_qasm, t)
    print(f"完整电路已生成至: {output_qasm}")

完整电路已生成至: diverging_flow_t=pi_over_2_test_4.qasm
